In [13]:
import jsonlines
import spacy
import spacy.language
from spacy.tokens import Doc
from spacy.scorer import Scorer
from spacy.vocab import Vocab
import statsmodels
import pandas as pd

In [3]:
# path to jsonl overlap files
path_coco = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_coco.jsonl"
path_graf = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_leo.jsonl"
path_hoff = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_jona.jsonl"
path_jthn = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_jonathan.jsonl"

In [11]:
def jsonl_to_list(path):
    """takes path to jsonl file and returns list of dicts"""
    
    with jsonlines.open(path) as reader:
        list_of_dicts = list(reader)

    return list_of_dicts

In [133]:
def label_per_token(recipe):
    """takes annotated recipes as input and returns a dict that maps label to every token"""

    amount_tokens = len(recipe["tokens"])

    all_token_dict = { tuple(range(token["start"], token["end"]+1)) : "None" for token in recipe["tokens"]}    # each token as list and value "None"

    for span in recipe["spans"]:

        all_chars = list(range(span["start"], span["end"]+1))

        label = span["label"]

        for char in all_chars:              #compare all characters with all token characters
            for tok in all_token_dict.keys():
                if char in tok: 
                    all_token_dict[tok] = label
    
    #print(f"Amount tokens: {amount_tokens} vs. Length dict: {len(all_token_dict.keys())}")

    return all_token_dict #{token: label}

In [134]:
def token_table_per_recipe(recipe):
    """takes example annotated recipe and creates empty dict with token_start_char as indices and ent classes as column labels"""

    columns = ["Tokens", "Z", "TOOL", "V", "ATTR", "PRÄP", "ZEITP", "DAUER", "TEMP", "None"]

    token_table = pd.DataFrame(columns=columns)

    toks = [tuple(range(token["start"], token["end"]+1)) for token in recipe["tokens"]]

    token_table["Tokens"] = toks

    token_table.fillna(0, inplace=True)

    return token_table  #pd style table

In [135]:
def calculate_kappa(table):
    """takes table with tokens and labels as input and returns kappa"""

    from statsmodels.stats.inter_rater import fleiss_kappa 

    
    kappa = statsmodels.stats.inter_rater.fleiss_kappa(table, method="fleiss")


    return kappa

In [136]:
# list of individual overlap dicts
ov_recipes_coco = jsonl_to_list(path_coco)
ov_recipes_graf = jsonl_to_list(path_graf)
ov_recipes_hoff = jsonl_to_list(path_hoff)
ov_recipes_jthn = jsonl_to_list(path_jthn)

In [137]:
# create dict for each annotator: key=text of recipe, value = ent_set
ov_dict_coco = {example["text"] : label_per_token(example) for example in ov_recipes_coco}
ov_dict_graf = {example["text"] : label_per_token(example) for example in ov_recipes_graf}
ov_dict_hoff = {example["text"] : label_per_token(example) for example in ov_recipes_hoff}
ov_dict_jthn = {example["text"] : label_per_token(example) for example in ov_recipes_jthn}

In [138]:
ov_all_dicts = [ov_dict_coco, ov_dict_jthn, ov_dict_graf, ov_dict_hoff]
annot_names = ["Coco", "Giov", "Graf", "Hoff"]

In [139]:
#create list with empty df per recipe 
recipe_table_dict = {example["text"] : token_table_per_recipe(example) for example in ov_recipes_coco}

In [142]:
for person in ov_all_dicts:                 #loop through all annotators
    for example_recipe in person.keys():    #loop through all recipes of each annotator example_recipe = text
        
        try: 
            dataframe = recipe_table_dict[example_recipe]

            for key in person[example_recipe].keys():  #loop through label_per_tok
                        
                    i = dataframe.index[dataframe["Tokens"] == key]
                    col_i = dataframe.columns.get_loc(person[example_recipe][key])
                    dataframe.iloc[i, col_i] += 1

        except: 
            pass
            #print("Recipe was skipped.")


Recipe was skipped.
Recipe was skipped.
Recipe was skipped.
Recipe was skipped.
Recipe was skipped.
Recipe was skipped.
Recipe was skipped.
Recipe was skipped.


In [157]:
recipe_table_dict
list = []
for recipe in recipe_table_dict.values():
    if recipe[["Z", "TOOL", "V", "ATTR", "PRÄP", "ZEITP", "DAUER", "TEMP", "None"]].iloc[0].sum() == 4:
        list.append(recipe)
    else:
        print("skip")
print(len(list))

skip
skip
skip
24


In [158]:
#concat
df_con = pd.concat(list, ignore_index=True)

In [159]:
# drop column
df_con_dropped = df_con.drop(["Tokens"], axis=1)

In [161]:
df_con_dropped

,Z,TOOL,V,ATTR,PRÄP,ZEITP,DAUER,TEMP,None
0,0,0,0,0,0,0,0,0,4
1,0,0,0,0,0,0,0,0,4
2,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,4
4,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...
2852,0,0,4,0,0,0,0,0,0
2853,0,0,0,0,0,0,0,0,4
2854,0,0,0,1,0,3,0,0,0
2855,0,0,4,0,0,0,0,0,0


In [201]:
#token-level kappa
kappa = fleiss_kappa(df_con_dropped, method="fleiss")
print(f"Overall: {round(kappa, 3)}")

Overall: 0.949


In [202]:
import numpy as np
entities = ["Z", "TOOL", "V", "ATTR", "PRÄP", "ZEITP", "DAUER", "TEMP"]
for entity in entities: 
    df2 = df_con_dropped.iloc[:, np.r_[df_con_dropped.columns.get_loc(entity), df_con_dropped.columns.get_loc("None")]]
    #print(df2)
    for i in range(len(df2)):
        if df2.iloc[i, :].sum() != 4:
            missing = 4 - (df2.iloc[i, 0] + df2.iloc[i, 1])
            df2.iloc[i, 1] += missing
    #print(df2)
    kap = fleiss_kappa(df2)
    print(f"{entity}: {round(kap, 3)}")

Z: 0.963
TOOL: 0.939
V: 0.992
ATTR: 0.886
PRÄP: 0.974
ZEITP: 0.865
DAUER: 0.96
TEMP: 0.855
